# Script to find MHD modes

In [1]:
import numpy as np
import scipy.special, scipy.optimize
from py_vlasov.dispersion_tensor import dt_wrapper
from py_vlasov.util import (real_imag, list_to_complex)
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named 'py_vlasov'

In [3]:
import os

In [4]:
os.getenv('PYTHONPATH')

###### To find the dispersion relation of Alfven or slow mode, we first adjust beta, then adjust angle, and finally deal with the right k

###### seed

###### angle = 85 k = 0.1 beta = 0.2 freq = 0.01190802-0.00635779j

In [ ]:
mass_ratio = 100
seed_angle = 85
seed_beta = 0.2
seed_k = 0.1
seed_freq = 0.01190802-0.00635779j # slow mode
#seed_freq = 0.26147871 -0.00747444j # fast mode
#seed_freq = 1.95444741e-02 -6.32084029e-06j # alfven mode
target_angle = 89.95
target_beta = 1.
target_k = 4.

###### now lets first adjust angle to 89.9 degrees

In [ ]:
beta = seed_beta
k = seed_k
theta = seed_angle
start = seed_angle
stop = target_angle
num = 20
angle_list_1 = np.linspace(start, stop, num, endpoint=True)
print(angle_list_1)

In [ ]:
theta = seed_angle
theta = theta * np.pi/180.
result_1 = [seed_freq]
for i in list(range(num))[1:]:
    theta = angle_list_1[i]
    theta = theta * np.pi/180.
    kz = k * np.cos(theta)
    kp = k * np.sin(theta)
    f = lambda wrel: real_imag(dt_wrapper(wrel[0] + 1j * wrel[1], kp, kz, beta, tetp = 1, method = 'numpy', mratio=mass_ratio))
    prev_angle = angle_list_1[i-1]
    guess = result_1[-1] * np.cos(theta)/ np.cos( prev_angle * np.pi/180)
    freq = scipy.optimize.fsolve(f, real_imag(guess))
    result_1 += [list_to_complex(freq)]

In [ ]:
result_1

###### then we adjust beta value

In [ ]:
theta = target_angle
theta = theta * np.pi/180
kz = 0.1 * np.cos(theta)
kp = 0.1 * np.sin(theta)
beta = seed_beta

In [ ]:
start = np.log10(seed_beta)
stop = np.log10(target_beta)
num = 10
beta_list = np.logspace(start, stop, num, endpoint = True)
print(beta_list)

In [ ]:
result_2 = [result_1[-1]]
for beta in beta_list[1:]:
    f = lambda wrel: real_imag(dt_wrapper(wrel[0] + 1j * wrel[1], kp, kz, beta, tetp = 1, method = 'numpy', mratio=mass_ratio))
    freq = scipy.optimize.fsolve(f, real_imag(result_2[-1]))
    print(result_2[-1])
    result_2 += [list_to_complex(freq)]

###### then we extend to larger k value

In [ ]:
beta = target_beta
theta = target_angle
theta = theta * np.pi/180

In [ ]:
start = np.log10(seed_k)
stop = np.log10(target_k)
num = 20
klist = np.logspace(start, stop, num=num, endpoint=True)
print(klist)

In [ ]:
result_3 = [result_2[-1]]
for k in klist[1:]:
    kz = k * np.cos(theta)
    kp = k * np.sin(theta)
    f = lambda wrel: real_imag(dt_wrapper(wrel[0] + 1j * wrel[1], kp, kz, beta, tetp = 1, method = 'numpy', mratio=mass_ratio))
    freq = scipy.optimize.fsolve(f, real_imag(result_3[-1]))
    result_3 += [list_to_complex(freq)]

In [ ]:
result_3

###### normalize to dimensionless frequency: $\bar{\omega}\equiv \omega/k_z v_A$

In [ ]:
theta = target_angle * np.pi/180
kz = target_k * np.cos(theta)
print(kz)
w = result_3[-1]
print(w)
w_bar = w/kz * np.sqrt(target_beta)
print(w_bar)

### Reference: Alfven mode guess

In [ ]:
theta = target_angle * np.pi/180
kz = target_k * np.cos(theta)
kp = target_k * np.sin(theta)
beta = target_beta
alfven = kz/np.sqrt(beta)
print(alfven)

In [ ]:
f = lambda wrel: real_imag(dt_wrapper(wrel[0] + 1j * wrel[1], kp, kz, beta, tetp = 1, method = 'numpy'))
freq = scipy.optimize.fsolve(f, [alfven, 0])
print(freq)